In [1]:
import warnings
warnings.filterwarnings('ignore')
#load traning data
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.datasets import make_regression
import xgboost as xgb
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
from sklearn import ensemble
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from lightgbm import LGBMRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
import csv
import datetime
import os.path
from os import path
from datetime import datetime
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
import pickle


from StackingAveragedModels import StackingAveragedModels

In [2]:
REMOVE_ONLY_2_Outliers = True
USE_DUMMY_CAT_FEATURES = True
STACK_MODELLING = True
RANDOM_SEED = 42

In [3]:
df_X_train = pd.read_csv('./data/train_X_clean.csv')
df_X_test =  pd.read_csv('./data/test_X_clean.csv')
df_y_train = pd.read_csv('./data/train_y_clean.csv')

display(df_X_train.head(1))
display(df_X_test.head(1))
display(df_y_train.head(1))
print(df_X_train.columns)

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleCondition_Normal,SaleCondition_Partial,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_Oth,SaleType_WD
0,11.693,11.686,0.000,0.730,1.541,1.820,1.541,11.170,0.000,1.194,...,1,0,0,0,0,0,0,0,0,1


,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleCondition_Normal,SaleCondition_Partial,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_Oth,SaleType_WD
0,11.819,0.000,0.000,0.730,1.194,1.820,1.541,10.105,7.397,2.056,...,1,0,0,0,0,0,0,0,0,1


,SalePrice
0,12.248


Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'Alley', 'BedroomAbvGr',
       'BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFinType1',
       ...
       'SaleCondition_Normal', 'SaleCondition_Partial', 'SaleType_COD',
       'SaleType_CWD', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI',
       'SaleType_ConLw', 'SaleType_Oth', 'SaleType_WD'],
      dtype='object', length=215)


In [4]:
X = df_X_train.iloc[:, 0:].values 
print(X.shape)
#print(X[0])

y = df_y_train.iloc[:, 0:].values 
print(y.shape)
y = df_y_train.values.reshape(df_y_train.values.shape[0],) 
print(y)

(1458, 215)
(1458, 1)
[12.24769912 12.10901644 12.31717117 ... 12.49313327 11.86446927
 11.90159023]


In [5]:
#Validation function
n_folds = 5

kfolds = KFold(n_splits=n_folds, shuffle=True, random_state=42)


# rmsle
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))


# build our model scoring function
def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)

In [6]:
# setup models    
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

In [7]:
#lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=1e7, alphas=alphas2, random_state=42, cv=kfolds))
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [8]:
#elasticnet = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e7, alphas=e_alphas, 
#                                                        cv=kfolds, random_state=42, l1_ratio=e_l1ratio))
elasticnet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [9]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

In [10]:
gbr = ensemble.GradientBoostingRegressor(n_estimators=3000, learning_rate=0.04, max_depth=3, max_features='sqrt',
                                min_samples_leaf=12, min_samples_split=10, loss='huber', random_state =42)

In [ ]:
xgboost = xgb.XGBRegressor(colsample_bytree=0.4, gamma=0.4,learning_rate=0.03, max_depth=3, min_child_weight=1.5,
                           n_estimators=5000, reg_alpha=0.45, reg_lambda=0.85,subsample=0.5, seed=42, silent=1, 
                           random_state =7, nthread = -1) 

In [ ]:
lightgbm = LGBMRegressor(objective='regression', num_leaves=4,learning_rate=0.03, n_estimators=5000,
                         max_bin=100, bagging_fraction=0.75,bagging_freq=5, bagging_seed=7,feature_fraction=0.2,
                         feature_fraction_seed=7,verbose=-1)

In [ ]:
ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt, cv=kfolds,))

In [ ]:
svr = make_pipeline(RobustScaler(),
                      SVR(C= 20, epsilon= 0.008, gamma=0.0003,))

In [ ]:
rf = RandomForestRegressor(random_state=RANDOM_SEED)

In [ ]:
svr = SVR(kernel = 'rbf',gamma='auto')

In [ ]:
#stack_gen = StackingCVRegressor(regressors=(ridge, lasso, svr, lightgbm, gbr, xgboost, rf), 
#                                meta_regressor=xgboost,use_features_in_secondary=True)

stack_gen = StackingAveragedModels(base_models = (elasticnet, gbr, KRR), meta_model = lasso)


In [ ]:
score = cv_rmse(stack_gen)

In [ ]:
print('TEST score on CV')

In [ ]:
score = cv_rmse(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

In [ ]:
score = cv_rmse(elasticnet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = cv_rmse(KRR)
print("\nKernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

In [ ]:
score = cv_rmse(gbr)
print("\nGradientBoosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

In [ ]:
score = cv_rmse(xgboost)
print("\nXgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

In [ ]:
score = cv_rmse(lightgbm)
print("\nLightgbm score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

In [ ]:
from StackingAveragedModels import StackingAveragedModels
score = cv_rmse(stack_gen)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [ ]:
score = cv_rmse(svr)
print("\nSVR score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

In [ ]:
score = cv_rmse(rf)
print("\nRandomForestRegressor score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

In [ ]:
print('START Fit')
print(datetime.now(), 'StackingCVRegressor')
stack_gen_model = stack_gen.fit(X, y)
# save the model to disk
filename = 'stack_gen.sav'
pickle.dump(stack_gen_model, open(filename, 'wb'))

print(datetime.now(), 'lasso')
lasso_model_full_data = lasso.fit(X, y)
filename = 'lasso.sav'
pickle.dump(lasso_model_full_data, open(filename, 'wb'))

print(datetime.now(), 'ridge')
ridge_model_full_data = ridge.fit(X, y)
filename = 'ridge.sav'
pickle.dump(ridge_model_full_data, open(filename, 'wb'))

print(datetime.now(), 'svr')
svr_model_full_data = svr.fit(X, y)
filename = 'svr.sav'
pickle.dump(svr_model_full_data, open(filename, 'wb'))

print(datetime.now(), 'GradientBoosting')
gbr_model_full_data = gbr.fit(X, y)
filename = 'gbr.sav'
pickle.dump(gbr_model_full_data, open(filename, 'wb'))


print(datetime.now(), 'xgboost')
xgb_model_full_data = xgboost.fit(X, y)
filename = 'xgboost.sav'
pickle.dump(xgb_model_full_data, open(filename, 'wb'))

print(datetime.now(), 'lightgbm')
lgb_model_full_data = lightgbm.fit(X, y)
filename = 'lightgbm.sav'
pickle.dump(lgb_model_full_data, open(filename, 'wb'))


print(datetime.now(), 'RandomForestRegressor')
rf_model_full_data = rf.fit(X, y)
filename = 'rf.sav'
pickle.dump(rf_model_full_data, open(filename, 'wb'))

In [ ]:
stacked_train_pred = stack_gen_model.predict(X)
print(rmsle(y, stacked_train_pred))

In [ ]:
gbr_train_pred = gbr_model_full_data.predict(X)
print(rmsle(y, gbr_train_pred))

In [ ]:
xgb_train_pred = xgb_model_full_data.predict(X)
print(rmsle(y, xgb_train_pred))

In [ ]:
lgb_train_pred = lgb_model_full_data.predict(X)
print(rmsle(y, lgb_train_pred))

In [ ]:
'''RMSE on the entire Train data when averaging'''

print('RMSLE score on train data:')
print(rmsle(y,stacked_train_pred*0.70 +
               gbr_train_pred*0.15 + lgb_train_pred*0.15 ))

In [ ]:
stacked_pred = np.expm1(stack_gen_model.predict(df_X_test.values))
xgb_pred = np.expm1(xgb_model_full_data.predict(df_X_test.values))
lgb_pred = np.expm1(lgb_model_full_data.predict(df_X_test.values))
gbr_pred = np.expm1(gbr_model_full_data.predict(df_X_test.values))

In [ ]:
#ensemble = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15 - scored 0.11665
ensemble = stacked_pred*0.70 + gbr_pred*0.15 + lgb_pred*0.15 ## scored 0.11576
#ensemble = stacked_pred*0.0 + gbr_pred*1 + lgb_pred*0.0 ## scored n scored 0.12423
#ensemble = stacked_pred*1.0 + gbr_pred*0 + lgb_pred*0.0 ## scored n scored 0.11600
#ensemble = stacked_pred*0.0 + gbr_pred*0 + lgb_pred*1.0 ## scored n scored ? 0.12147
submission = pd.read_csv("./data/sample_submission.csv")
sub = pd.DataFrame()
sub['Id'] = submission['Id'].values
sub['SalePrice'] = ensemble
sub.to_csv('submission.csv',index=False)

In [ ]:

model_weights = {}
model_weights = {'rf':0.01,'lasso':0.15, 'ridge':0.03, 'svr':0.01, 'gbr':0.2, 'xgb':0.15, 'lgb':0.15, 'stack':0.3}

def blend_models_predict(X_pred):
    return ((model_weights['rf'] * rf_model_full_data.predict(X_pred)) + \
            (model_weights['lasso'] * lasso_model_full_data.predict(X_pred)) + \
            (model_weights['ridge'] * ridge_model_full_data.predict(X_pred)) + \
            (model_weights['svr'] * svr_model_full_data.predict(X_pred)) + \
            (model_weights['gbr'] * gbr_model_full_data.predict(X_pred)) + \
            (model_weights['xgb'] * xgb_model_full_data.predict(X_pred)) + \
            (model_weights['lgb'] * lgb_model_full_data.predict(X_pred)) + \
            (model_weights['stack'] * stack_gen_model.predict(X_pred)))


print('RMSLE score on train data:')
print(X.shape)
print(y.shape)
#print(y.shape)
#print(rmsle(y, blend_models_predict(X)))

In [ ]:
X_test = df_X_test.iloc[:, 0:].values 
print(X_test.shape)
#print(X_test[0:])
print("Shape of output before prediction is {0}".format(y_test.shape))
print('Predict submission', datetime.now(),)
submission = pd.read_csv("./data/sample_submission.csv")
print(submission.columns)
submission.drop('SalePrice',axis=1)
print("Shape of submission is {0}".format(submission.shape))
print(submission.columns)
#submission.iloc[:,1] = np.floor(np.expm1(blend_models_predict(X_test)))
#y_test = np.floor(np.expm1(blend_models_predict(X_test[0])))
y_test = np.floor(np.expm1(xgb_model_full_data.predict(X_test)))### Working code
#y_test = np.floor(np.expm1(blend_models_predict(X_test)))

print("Shape of output is {0}".format(y_test.shape))
print(y_test)
#y_test_reshaped = y_test.reshape(-1,1)

submission.iloc[:,1] = y_test
print(submission.head())
print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
dataset = pd.DataFrame({'SalePrice':[ y_test]})

print(y_test[0:5])
print(len(y_test[0:2]))


result = pd.concat([submission['Id'], dataset], axis=1, sort=False)
print(result.head())
#result.to_csv("submission.csv", index=False)
print('Save submission', datetime.now(),)